# CPLEX Interface testing

In [2]:
import numpy as np
import scipy as sp

### Converting to standard form test

In [165]:
f2 = np.array([4,2,7,5]).reshape(4,1)
A2 = np.random.randint(0,10,3*4).reshape(3,4) # 3 ineq constraints
b2 = np.random.randint(50,100,3).reshape(3,1)
b2[1,0]=-b2[1,0]
Aeq2 = np.random.randint(1,2,1*4).reshape(1,4) # 1 ineq constraint
beq2 = np.random.randint(75,125,1).reshape(1,1)
lb2 = np.array([-10,0,10,-100]).reshape(4,1)
ub2 = np.array([np.inf, np.inf, 100,50]).reshape(f2.shape)
cont2 = np.array([1,0,1,0]).reshape(f2.shape)
#print(f2, A2,b2, Aeq2, beq2, lb2, ub2, intcon2)

In [138]:
f2 = -np.array([1,2,3,4,5]).reshape(5,1);
Aeq2 = np.array([3,2,0,0,0]).reshape(1,5);
beq2 = np.array([[5]]);
A2 = np.array([[0, 1, 1, 0, 0],
              [0, 0, 1, 1, 0],
              [0, 0, 0, 1, 1],
              [1, 0, 0, 0, 1]]);

b2 = np.array([5,4,3,2]).reshape(4,1)

lb2 = np.array([0, -np.inf, -3, 3, 0]).reshape(5,1)
ub2 = np.array([np.inf, np.inf, 100, 100, np.inf]).reshape(5,1)

cont2 = np.array([1,0,1,0,1]).reshape(f2.shape)

In [166]:
lb_count = 0
ub_count = 0
nvar = np.size(f2)
# Defining Alb, Aub, blb and bub to convert bound constraints to standard form
Aub = np.zeros((np.sum(ub2<np.inf),nvar))
temp = np.sum(np.all((lb2!=0, lb2>-np.inf),axis=0)) #Number of lower bound constraints
Alb = np.zeros((temp,nvar))
bub = np.zeros((np.sum(ub2<np.inf),1))
blb = np.zeros((temp,1))        

In [167]:
for i in np.arange(nvar):
    # Converting bound constraints to <= constraints
    # Ensure all variables satisfy >= 0
    if ub2[i, 0] < np.inf:
        # If upper bound is finite, then a constraint has to be added in standard form
        Aub[ub_count, i] = 1
        bub[ub_count, 0] = ub2[i]
        ub_count = ub_count + 1
    if lb2[i,0] < 0:
        # If lower bound is less than 0, a constraint has to be added
        # In fact a new variable has to be added. So the sizes of all the
        # other entities have to be changed.
        # i.e., rewriting x = x1 + x2 with x1 >=0 ,x2 >= 0
        f2 = np.vstack((f2, [[-f2[i,0]]])) #Corresponding "-c_i" for new term in obj
        cont2 = np.vstack((cont2, [[cont2[i,0]]])) 
        t1 = A2.shape[0] # Number of inequality constraints
        t2 = Aeq2.shape[0] # Number of equality constraints
        t3 = Aub.shape[0] # Number of upperbound constraints
        t4 = Alb.shape[0] # Number of lowerbound constraints
        # Adding a column to constraint matrices with sign flipped, for the new variable
        A2 = np.hstack((A2,
                    -A2[:,i].reshape(t1,1)
                    ))
        Aeq2 = np.hstack((Aeq2,
                    -Aeq2[:,i].reshape(t2,1)
                    ))
        Aub = np.hstack((Aub,
                    -Aub[:,i].reshape(t3,1)
                    ))
        Alb = np.hstack((Alb,
                    -Alb[:,i].reshape(t4,1)
                    ))
        print(i, np.size(f2),Alb.shape)
        if lb2[i,0]>-np.inf:
            # However if lower bound is not -Inf (i.e., finite), then that constraint is 
            # added
            Alb[lb_count, i] = -1
            Alb[lb_count, np.size(f2)-1] = 1 #last variable so far
            blb[lb_count, 0] = -lb2[i,0]
            lb_count = lb_count+1
    else:
        if lb2[i,0] > 0:
            # If lower bound is greater than 0, constraint has to be added
            Alb[lb_count,i] = -1
            blb[lb_count, 0] = -lb2[i,0]
            lb_count = lb_count+1

0 5 (3, 5)
3 6 (3, 6)


In [168]:
print(Aeq2, beq2, Alb,blb, Aub,bub, A2,b2)
# Count the number of constraints
nineq = np.size(b2) # Number of straight forward inequalities
neq = np.size(beq2) # Number of straight forward equalities
nlb = np.size(blb) # Number of lower bound inequalities
nub = np.size(bub) # Number of upper bound inequalities
# Now convert inequality constraints to equality constraints
# by adding slack variables (identity matrices to ineq constraints)
A2 = np.hstack((A2, np.identity(nineq), np.zeros((nineq, nlb+nub))))
Alb = np.hstack((Alb, np.zeros((nlb,nineq)),np.identity(nlb),np.zeros((nlb,nub))))
Aub = np.hstack((Aub, np.zeros((nub,nineq+nlb)),np.identity(nub)))
Aeq2 = np.hstack((Aeq2, np.zeros((neq, nineq+nlb+nub))))
# Giving "0" in the objectives for slack variables
f2 = np.vstack((f2, np.zeros((nineq+nlb+nub,1))))
# Slack variables are assumed to be continuous
# All input matrices are integers, this can be strengthened and slacks can be called
# as integer variables. Perhaps that can give stronger cuts!
cont2 = np.vstack((cont2, np.ones((nineq+nlb+nub,1))))
print(Aeq2, beq2, Alb,blb, Aub,bub, A2,b2)

[[ 1  1  1  1 -1 -1]] [[89]] [[-1.  0.  0.  0.  1. -0.]
 [ 0.  0. -1.  0. -0. -0.]
 [ 0.  0.  0. -1. -0.  1.]] [[  10.]
 [ -10.]
 [ 100.]] [[ 0.  0.  1.  0. -0. -0.]
 [ 0.  0.  0.  1. -0. -1.]] [[ 100.]
 [  50.]] [[ 3  5  1  8 -3 -8]
 [ 5  0  4  3 -5 -3]
 [ 2  6  1  4 -2 -4]] [[ 99]
 [-70]
 [ 78]]
[[ 1.  1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]] [[89]] [[-1.  0.  0.  0.  1. -0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -0. -0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1. -0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]] [[  10.]
 [ -10.]
 [ 100.]] [[ 0.  0.  1.  0. -0. -0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1. -0. -1.  0.  0.  0.  0.  0.  0.  0.  1.]] [[ 100.]
 [  50.]] [[ 3.  5.  1.  8. -3. -8.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 5.  0.  4.  3. -5. -3.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 2.  6.  1.  4. -2. -4.  0.  0.  1.  0.  0.  0.  0.  0.]] [[ 99]
 [-70]
 [ 78]]


In [169]:
# Creating the final A and b matrices for the standard form
Afin = np.vstack((Aeq2, Alb, Aub, A2))
bfin = np.vstack((beq2, blb, bub, b2))
# Ensure b in standard form is non-negative

In [170]:
negb = np.where(bfin<0)[0]
print(A2,b2)
Afin[negb,:] = -Afin[negb,:]
bfin[negb,:] = -bfin[negb,:]
print(Afin,bfin)

[[ 3.  5.  1.  8. -3. -8.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 5.  0.  4.  3. -5. -3.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 2.  6.  1.  4. -2. -4.  0.  0.  1.  0.  0.  0.  0.  0.]] [[ 99]
 [-70]
 [ 78]]
[[ 1.  1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1. -0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-0. -0.  1. -0.  0.  0. -0. -0. -0. -0. -1. -0. -0. -0.]
 [ 0.  0.  0. -1. -0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0. -0. -0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1. -0. -1.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 3.  5.  1.  8. -3. -8.  1.  0.  0.  0.  0.  0.  0.  0.]
 [-5. -0. -4. -3.  5.  3. -0. -1. -0. -0. -0. -0. -0. -0.]
 [ 2.  6.  1.  4. -2. -4.  0.  0.  1.  0.  0.  0.  0.  0.]] [[  89.]
 [  10.]
 [  10.]
 [ 100.]
 [ 100.]
 [  50.]
 [  99.]
 [  70.]
 [  78.]]


###  Cplex - Python MIP form interface

In [171]:
import cplex

In [172]:
M = cplex.Cplex()
M.objective.set_sense(M.objective.sense.minimize)
print(f2.squeeze().tolist())
M.variables.add(obj=f2.squeeze().tolist())
nCons = np.size(bfin)
nVar = np.size(f2)

[4.0, 2.0, 7.0, 5.0, -4.0, -5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [173]:
LHS = [[range(nVar), Afin[i,:].copy().squeeze().tolist()] for i in range(nCons)]
M.linear_constraints.add(lin_expr = LHS, senses='E'*nCons, rhs=bfin.squeeze().tolist())

[[range(0, 14), [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [range(0, 14), [-1.0, 0.0, 0.0, 0.0, 1.0, -0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]], [range(0, 14), [-0.0, -0.0, 1.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0]], [range(0, 14), [0.0, 0.0, 0.0, -1.0, -0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]], [range(0, 14), [0.0, 0.0, 1.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]], [range(0, 14), [0.0, 0.0, 0.0, 1.0, -0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]], [range(0, 14), [3.0, 5.0, 1.0, 8.0, -3.0, -8.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [range(0, 14), [-5.0, -0.0, -4.0, -3.0, 5.0, 3.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]], [range(0, 14), [2.0, 6.0, 1.0, 4.0, -2.0, -4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]]


range(0, 9)

In [203]:
M.parameters.lpmethod.set(M.parameters.lpmethod.values.primal)
M.parameters.preprocessing.presolve.set(0)
M.solve()

CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0


In [246]:
print(M.solution.status[M.solution.get_status()])
print(M.solution.get_objective_value())
print(np.array(M.solution.get_values())*9)

infeasible
316.0000000000001
[    0.   815.  2360.     0.  1474.   900.  6078.     0.     0. -1384.
  2270.     0. -1460.  1350.]


In [240]:
print(np.around(9*np.array(M.solution.advanced.binvarow()), 10))
print(M.variables.get_num())
print(M.linear_constraints.get_num())

[[ -0.   9.   0.   0.   0.   0.   0.  -1.   1.   0.   0.   4.   0.   0.]
 [ -0.   0.   9.   0.   0.   0.   0.  -4.  -5.   0.   0.  -2.   0.   0.]
 [ -9.   0.   0.   0.   9.   0.   0.  -5.  -4.   0.   0.  -7.   0.   0.]
 [  0.   0.   0.  -9.   0.   9.   0.   0.   0.   0.   0.   9.   0.   0.]
 [ -0.   0.   0.   0.   0.   0.   9.  -6. -12.   0.   0.  33.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   5.   4.   9.   0.   7.   0.   0.]
 [ -0.   0.   0.   0.   0.   0.   0.  -4.  -5.   0.   9.  -2.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   4.   5.   0.   0.   2.   9.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   9.   0.   9.]]
14
9


In [248]:
M.variables.get_num()

14

In [5]:
temp = np.random.randint(0,2,(10,))
print(temp)

[0 1 1 0 0 1 1 0 1 1]


In [7]:
temp2 = ['I' if i==1 else 'C' for i in temp]
print(temp2)

['C', 'I', 'I', 'C', 'C', 'I', 'I', 'C', 'I', 'I']


In [10]:
temp3 = [0 if i=='C' else 1 for i in temp2]
print(np.array(temp3))

[0 1 1 0 0 1 1 0 1 1]
